In [ ]:
!pip install --upgrade gensim
from re import sub
from gensim.utils import simple_preprocess
import numpy as np
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity
# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(glove)

!pip install contractions
!pip install googletrans==4.0.0-rc1
import nltk
import pandas as pd
import contractions
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from googletrans import Translator
translator = Translator()

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('averaged_perceptron_tagger')
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# reading .csv dataset, cleaned manually in order to remove greetings and PROPER NOUN at the beginning and at the end of the email
import pandas as pd
data = pd.read_csv("put_csv_file_here.csv")


In [ ]:
#Translate text to English
data['Translated'] = data['Body'].apply(lambda x: translator.translate(x, dest='en').text)

#Extend Contractions
data["Extended"] = data['Translated'].apply(lambda x: contractions.fix(x))

In [ ]:
#Sort by month if needed
month_sort = data.loc[data.Date.str.contains('May', na=False)]    #Sep, Oct, Nov, Dec, Jan, Feb, Mar, Apr, May, Jun, Jul, Aug
text = month_sort["Extended"].to_list()
print(text)

In [ ]:
#without sorting by months
text = data["Extended"].to_list()
print(text)

In [ ]:
#preprocessing and cleaning
import re
import string
from six.moves.html_parser import HTMLParser
h = HTMLParser()
target = string.printable + "öäüÖÄÜ"

def clean(value):
  sentences = []
  for v in text:
    if type(v) is not str:
        continue
    doc = re.sub ("\n", " ", v)
    doc = re.sub ("\r", " ",doc)
    doc = re.sub("http\S+", " ", doc) #removing urls
    doc = re.sub("[`~!@#$%^&*()_|+\-=?;:'<>\{\}\[\]\\\/]", ' ', doc)
    doc = h.unescape(doc) #converting other HTML entities to recongisable characters
    doc = re.sub("(\s|\t){2,}", " ", doc) #removing unnecessary spaces
    doc = re.sub("\S*@\S*\s?", " ", doc) #removing emails
    doc = re.sub("@[^\d]", " ", doc) #removing phone numbers
    sentences.append(doc.strip()) #removing leading and trailing spaces and adding clean string to the list
  return sentences


clean_sents = clean(text)
print("clean_sents")
print(len(clean_sents))
print(clean_sents)

def split_sentences (cleaned_sents):
    doc = nlp(str(clean_sents))
    splitted_sent = []
    for sent in doc.sents:
        splitted = sent.text
        splitted_sent.append(splitted)
    print(len(splitted_sent))
    return splitted_sent

final_text = split_sentences(clean_sents)

def remove_duplicates(sents):
  unique_sents = []
  for sent in sents:
    if sent not in unique_sents:
      unique_sents.append(sent)
  return unique_sents

removed_dup = remove_duplicates(final_text)
print(len(removed_dup))

filtered_sent = [sent for sent in removed_dup if len(sent) > 10 and len(sent) < 100]
print(len(filtered_sent))

In [ ]:
d = dict()
d["domestic"] = ["esteem", "reputation", "oral format of information", "exemplary format of information" , "anecdotal information", "trust", "authority"]
d['industrial'] = ["productivity", "efficiency", "criteria", "statistics", "functional link", "professiional competency", "expertise"]
d['project'] = ["projects", "network expansion", "proliferation of links", "to coordinate", "to adjust to others", "engaged", "engaging", "enthusiastic", "involved", "flexible", "adaptable", "scalable", "employable", "autonomous", "project manager", "coach", "expert", "supplier", "innovator", "connection tools", "new technologies", "informal relationships", "trust relationships", "partnership", "outsourcing", "business networks", "network companies", "synapse", "authoritarian", "rigid", "network shutdown", "redistribution of connections", "insert into networks", "provide employability", "the end of a project", "the start of another project", "to be called upon to participate", "to involve", "to reject", "need to be linked", "network"]
d['market'] = ["price", "monetory format of information", "exchange", "desire", "purchasing power"]
d['civic'] = ["collective interest", "formal format of information", "official format of information", "solidarity", "equality"]
d['inspiration'] = ["grace", "nonconformity", "creativeness", "emotional format of information", "passion", "creativity", "ingenuity"]
d['opinion'] = ["renown", "semiotic format of information", "recognition", "celebrity"]


In [ ]:
#alternative extended list of dic
d= dict()
d["inspiration"]=['inspiration','unspeakable and ethereal','weird','unusual','marvellous','unspeakable','disturbing','exciting','spontaneous','emotional','concern for creation','love','passion','create','the enlightened','mind','fairy','shadow','freak','I','child','women','mad','artist','the waking dream','body','dream','unconscious','dope','escape from habits','question','risk','detour','the universal value of singularity','independent','genius','the alchemy of unexpected encounters','discover','to research','imagine','dream','explode','TO DO','the reality of the imaginary','imaginary','unconscious','uses','proprieties','adventure','quest','mental journey','lived experience','illumination','intuition','almost appear','vertigo','surpass oneself','masterpiece','hover','will have','fantasy','the certainty of intuition','symbol','signs']
d["domestic"]=['generation','hierarchy','tradition','hierarchical superiority','caring','well-mannered','easy','distinct','discreet','reserve','trustworthy','franc','faithful','ease of habit','fold','common sense','natural','character','the superiors and the inferiors','father','king','ascendants','parents','family','person','boss','chief','child','women','foreign','Single','I','Me','dog','cat','others','visitor','surroundings','neighbors','third','the rules of good manners','good manners','propriety','rank','title','remains','presentation','signature','announcement','gifts','flowers','rejection of selfishness','helpfulness','duty (and she)','harmony','respect and responsibility','authority','subordination','respectability','honor','shame','the trade of well-mannered people','reproduce','give birth','educate','invite','give','receive','render','recommend','thank','respect','the soul of the home','family','environment','principle','party','birth','death','marriage','worldliness','conversation','distinction','nomination','enjoy','congratulate','admonitions','report','the exemplary anecdote','give an example','prejudice']
d['project'] = ["projects", "network expansion", "proliferation of links", "to coordinate", "to adjust to others", "engaged", "engaging", "enthusiastic", "involved", "flexible", "adaptable", "scalable", "employable", "autonomous", "project manager", "coach", "expert", "supplier", "innovator", "connection tools", "new technologies", "informal relationships", "trust relationships", "partnership", "outsourcing", "business networks", "network companies", "synapse", "authoritarian", "rigid", "network shutdown", "redistribution of connections", "insert into networks", "provide employability", "the end of a project", "the start of another project", "to be called upon to participate", "to involve", "to reject", "need to be linked", "network"]							
d['industrial']=['efficiency','performance','coming','functional','efficient','reliable','operational','energy','work','professionals','experts','specialists','responsible','operator','tools','resources','method','stain','space','environment','axis','direction','criteria','definition','list','chart','plan','calendar','plan','objective','quantity','variable','series','mean','probability','nun','factor','cause','progress','investment','sympathy','detachment','mastery','function','implement','gear linkages','be a function of','cog','need','to condition','necessary relationship','to integrate','arrange','control','stabilize','to expect','implant','adapted','detect','analyze','take into account','determine','highlight','measure','formalize','standardization','to optimise','solve','to treat','organization','system','test','launch','getting started','Implementation','achievement','effective','correct','in working order','functioning','measure']
d['market'] =['competition','rivalry','competition','desirable','salable','value','millionaire','winner','hated','unsuitable','random','interest','desire','selfishness','competitors','business man','seller','customer','Buyer','self employed','richness','luxury items','opportunism','freedom','opening','pay attention to others','dynamic','emotional distance','take a step back','possess','to interest','buy','produce','sell','to be in business with','negotiate','benefit','monetize','pay','compete','market','affair','case settled','in the bag','bargain','price','justified value','reasonable value','true value','money','profit','result','retribution']
d['civic']=['the pre-eminence of collectives','collective','all','general will','regulatory and representative','unitary','legal','regulatory','official','representative','allowed','incumbent','free','the aspiration to civil rights','civil rights','political aspirations','participation','public authorities','left','federation','section','desk','committee','elected','representing','delegate','secretary','member','right','legislation','decree','arrangement','measure','courts','formality','procedure','protocol of agreement','exemption','electoral capacity','coded','criteria','riding','electoral list','program','orientation','statement','attach','brochure','newsletter','leaflet','slogan','seat','permanence','local','acronym','map','solidarity','exceed','to renouncer','struggle','delegation reports','membership','representation','delegation','translate aspirations','unify','mobilize','gather','exclude','join','rally','make a call','discuss','speak','inform','codify','legalize','empower','go to court','the democratic republic','Republic','state','democracy','base','electorate','representative institutions','parliament','assembly','Congress','advice','meeting','session','movement','manifest the presence','dispute','appeal','justice','the verdict of the ballot','rumor','noise','fashion','coast','resounding','vote','election','consultation','mobilization','measure audience','law','legal rules','statutes']
d['opinion']=['the reality of opinion','others','General public','Fame','famous','recognized','visible','hit','be successful','stand out','persuasive','catchy','the desire to be recognized','self-esteem','consideration','desire to','stars and supporters','a personality','opinion leader','spokesperson','relay','journalist','Press officer','names in the media','brand','message','transmitter','receiver','campaign','public relations','press','interview','communicated','support','brochure','mailing','badge','audio-visual','atmosphere','decor','reconcentration to secrecy','reveal','to be recognized','identify','identification','strength','persuasion','influence','convince','to raise awareness','hang','seduce','drill','capture','throw','issue','circulate','propagate','discuss','speak up','promote','orient','amplify','talk about','to quote','public image','audience','target','positioning','the presentation of the event','expression','press conference','imagination','open door','evidence of success','known','repercussion','reduce to','cause','rally to','awareness']

In [ ]:
for (name, v) in d.items():
  query_string = str(v)

  documents = filtered_sent

  stopwords = ['the', 'and', 'are', 'a', "/"]

  def preprocess(doc):
      # Tokenize, clean up input document string
      doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
      doc = sub(r'<[^<>]+(>|$)', " ", doc)
      doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
      doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
      return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

  # Preprocess the documents, including the query string
  corpus = [preprocess(document) for document in documents]
  query = preprocess(query_string)

  # Build the term dictionary, TF-idf model
  dictionary = Dictionary(corpus+[query])
  tfidf = TfidfModel(dictionary=dictionary)

  # Create the term similarity matrix.
  similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)


  # Compute Soft Cosine Measure between the query and the documents.
  query_tf = tfidf[dictionary.doc2bow(query)]

  index = SoftCosineSimilarity(
              tfidf[[dictionary.doc2bow(document) for document in corpus]],
              similarity_matrix)

  doc_similarity_scores = index[query_tf]


  # Output the sorted similarity scores and documents
  sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
  print(name)
  for idx in sorted_indexes:
      #similarity_results = f'\n {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}'
      if doc_similarity_scores[idx] > 0.1:
          similarity_results = f'\n {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}'

          print(similarity_results)
